![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/competitions/miad2024-12-prediccion-precio-vehiculos).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')
!python --version

Python 3.10.5


In [2]:
# Importación librerías
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn import metrics
# Librerías para Procesamiento de datos
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
# Librerías para Modelos
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import joblib

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)

Preprocesamiento de Datos
Tratamiento de Datos Dispersos

In [ ]:
dataTraining_2 = dataTraining.copy()
dataTesting_2 = dataTesting.copy()
# Ajustando precio
dataTraining_2['Price'] = np.log(dataTraining_2['Price'])
# Ajustando Milage
dataTraining_2['Mileage'] = np.log(dataTraining_2['Mileage'])
dataTesting_2['Mileage'] = np.log(dataTesting_2['Mileage'])

In [ ]:
# Aplicar label encoder en datos de Entrenamiento (fit_transform)

X = dataTraining_2[['Year', 'Mileage', 'State', 'Make', 'Model']]
le_State = LabelEncoder()
le_Make = LabelEncoder()
le_Model = LabelEncoder()
X['State_encoded'] = le_State.fit_transform(X.State)
X['Make_encoded'] = le_Make.fit_transform(X.Make)
X['Model_encoded'] = le_Model.fit_transform(X.Model)
X = X[['Year', 'Mileage', 'State_encoded', 'Make_encoded', 'Model_encoded']]
y = dataTraining_2['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=12)

# Aplicar label encoder en datos de Evaluación para Kaggle (transform)
data_X_Kaggle = dataTesting_2
data_X_Kaggle['State_encoded'] = le_State.transform(data_X_Kaggle.State)
data_X_Kaggle['Make_encoded'] = le_Make.transform(data_X_Kaggle.Make)
data_X_Kaggle['Model_encoded'] = le_Model.transform(data_X_Kaggle.Model)
data_X_Kaggle = data_X_Kaggle[['Year', 'Mileage', 'State_encoded', 'Make_encoded', 'Model_encoded']]

 Random Forest

In [8]:
# Entrenamiento (fit) y desempeño del modelo RF_Regressor(4,750,20)
reg_RF = RandomForestRegressor(max_features=4,n_estimators=100, max_depth=19, random_state=1, min_samples_split=17, n_jobs=-1)
reg_RF.fit(X_train, y_train)
y_pred_RF = reg_RF.predict(X_test)
# Guardar el modelo entrenado
joblib.dump(reg_RF, 'model_rf.pkl', compress=6)

['model_rf.pkl']

API

In [12]:
from flask import Flask, request, jsonify
import joblib
import numpy as np

# Cargar el modelo entrenado
model = joblib.load('model_rf.pkl')

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    # Extraer características desde el JSON
    features = [
        data['Year'],
        data['Mileage'],
        data['State_encoded'],
        data['Make_encoded'],
        data['Model_encoded']
    ]
    
    # Predecir el precio usando el modelo cargado
    price_log = model.predict([features])[0]  # Suponiendo que el modelo espera una matriz 2D
    price = np.exp(price_log)  # Convertir de log-precio a precio

    # Devolver la predicción como un JSON
    return jsonify({
        'predicted_price': price
    })

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.3:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1